In [31]:
import gym
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [32]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


https://github.com/openai/gym/blob/master/gym/spaces/box.py#L31

コードみたけどよくわからん

## OpenAI Gymの使い方

In [33]:
#　環境のreset
env.reset()

array([ 0.0449586 ,  0.01342331,  0.04773418,  0.02862372])

observationが返ってきます（observation? -> state(状態)のこと)

In [34]:
env.action_space

Discrete(2)

このゲームはアクションが２つあることがわかりました. 何を食わせばいいのかはわからない

In [35]:
env.observation_space

Box(4,)

このゲームはstateを４つ返します。

[position of cart, velocity of cart, angle of pole, rotation rate of pole]
-> [カートの位置、カートの速度、ポールの角度、ポールの回転数（？）]

In [36]:
action = 0
env.step(action)

(array([ 0.04522706, -0.18234953,  0.04830665,  0.335977  ]), 1.0, False, {})

actionを渡すことでゲームを進めます.

返り値は[observation, reward, done, info]です

* reward -> 報酬
* done -> Trueだとゲームクリアかゲームオーバー. ゲームオーバーした場合は環境をresetしなければいけない
* info -> 今回は使わない

## OpenAI Gymをランダムで動かしてみる

In [37]:
episodes = 10
for episode in range(episodes):
    done = False
    observatin = env.reset()
    while True:
        # 表示させたいがうまくいかない...
        # env.render()
        action = env.action_space.sample()
        observation, reward, done, _ = env.step(action)
        
        print(action, observation, reward, done)
        if done: break

1 [ 0.04802026  0.20630822 -0.007726   -0.3192784 ] 1.0 False
0 [ 0.05214643  0.01129715 -0.01411157 -0.02904195] 1.0 False
0 [ 0.05237237 -0.18361962 -0.01469241  0.25915543] 1.0 False
0 [ 0.04869998 -0.37852878 -0.0095093   0.54716819] 1.0 False
1 [ 0.0411294  -0.18327453  0.00143406  0.25150436] 1.0 False
0 [ 0.03746391 -0.37841693  0.00646415  0.54463928] 1.0 False
0 [ 0.02989557 -0.57362911  0.01735693  0.83935189] 1.0 False
1 [ 0.01842299 -0.3787484   0.03414397  0.55217753] 1.0 False
0 [ 0.01084802 -0.57433282  0.04518752  0.85541954] 1.0 False
1 [-0.00063863 -0.37985483  0.06229591  0.57728094] 1.0 False
0 [-0.00823573 -0.57579207  0.07384153  0.88891986] 1.0 False
0 [-0.01975157 -0.77183423  0.09161993  1.20387145] 1.0 False
1 [-0.03518825 -0.57800825  0.11569736  0.94125048] 1.0 False
1 [-0.04674842 -0.3846196   0.13452237  0.68704603] 1.0 False
0 [-0.05444081 -0.58132695  0.14826329  1.01887172] 1.0 False
0 [-0.06606735 -0.77808042  0.16864072  1.35419399] 1.0 False
0 [-0.08

ok

## 状態についての説明

今回もらう状態は連続値です。

In [52]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
df = pd.DataFrame([env.observation_space.high, env.observation_space.low],
    index=['high', 'low'],
    columns=['カートの位置', 'カートの速度', 'ポールの角度', 'ポールの回転数（？）'])

In [53]:
df

,カートの位置,カートの速度,ポールの角度,ポールの回転数（？）
high,4.8,3.402823e+38,0.418879,3.402823e+38
low,-4.8,-3.402823e+38,-0.418879,-3.402823e+38


連続値を状態として学習しても良いですが、すごく時間がかかりそうです。なので、ある程度グルーピングしてあげる -> 離散値に変換してあげます。

In [68]:
v = np.arange(-4.8, 4.8, 0.1)
vv = pd.DataFrame(v)
vv

,0
0,-4.8
1,-4.7
2,-4.6
3,-4.5
4,-4.4
5,-4.3
6,-4.2
7,-4.1
8,-4.0
9,-3.9


In [75]:
pd.qcut(v, 6).value_counts()

(-4.801, -3.217]    16
(-3.217, -1.633]    16
(-1.633, -0.05]     16
(-0.05, 1.533]      16
(1.533, 3.117]      16
(3.117, 4.7]        16
dtype: int64

イメージはこういうことをやります。データをいくつかまとめることで離散値にします。

In [50]:
# これが処理をおこなってくれます
from src.utils import to_discrete_value

# to_discrete_value(observation, 6)  # 第２引数はどれぐらい区切るか

上記例だと、6 ** 4の状態になるため、合計1296個の状態に削減することができます。